In [1]:
%reload_ext autoreload
%autoreload 2
from cudants.io.image import Image, BatchedImages
from cudants.registration.rigid import RigidRegistration
from cudants.registration.affine import AffineRegistration
import torch
from torch.optim import SGD, Adam
from torch.nn import functional as F
from torch import nn

In [2]:
# img1 = Image.load_file('/data/BRATS2021/training/BraTS2021_00598/BraTS2021_00598_t1.nii.gz')
# img2 = Image.load_file('/data/BRATS2021/training/BraTS2021_00597/BraTS2021_00597_t1.nii.gz')
img1 = Image.load_file('/data/BRATS2021/training/BraTS2021_00262/BraTS2021_00262_t1.nii.gz')
img2 = Image.load_file('/data/BRATS2021/training/BraTS2021_00403/BraTS2021_00403_t1.nii.gz')

# img1 = Image.load_file("images/atlas_2mm_1000_3.nii.gz")
# img2 = Image.load_file("images/atlas_2mm_1001_3.nii.gz")

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, Layout

def browse_image_slices(image_3d):
    # Infer the dimensions of image
    if len(image_3d.shape) == 3:
        dim_x, dim_y, dim_z = image_3d.shape
        color = False
    else:
        dim_x, dim_y, dim_z = image_3d.shape[:-1]
        color = True
    # Define a function to visualize the slices
    def plot_slice(x, y, z):
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))

        ax[0].imshow(image_3d[x, :, :], cmap='gray' if not color else None)
        ax[0].set_title('Slice at X = %d' % x)

        ax[1].imshow(image_3d[:, y, :], cmap='gray' if not color else None)
        ax[1].set_title('Slice at Y = %d' % y)

        ax[2].imshow(image_3d[:, :, z], cmap='gray' if not color else None)
        ax[2].set_title('Slice at Z = %d' % z)

        plt.show()

    # Use interact to create the UI with sliders
    interact(plot_slice,
             x=IntSlider(min=0, max=dim_x-1, value=dim_x//2, layout=Layout(width='600px')),
             y=IntSlider(min=0, max=dim_y-1, value=dim_y//2, layout=Layout(width='600px')),
             z=IntSlider(min=0, max=dim_z-1, value=dim_z//2, layout=Layout(width='600px')))

In [4]:
# Test the function
browse_image_slices(img1.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

In [5]:
# Test the function
browse_image_slices(img2.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

In [6]:
from cudants.registration.affine import AffineRegistration
fixed = BatchedImages([img1,])
moving = BatchedImages([img2,])
# transform = AffineRegistration([8, 4, 2, 1], [1000, 500, 250, 100], fixed, moving, \
transform = AffineRegistration([8, 4, 2, 1], [200, 100, 50, 20], fixed, moving, \
    loss_type='cc', optimizer='Adam', optimizer_lr=3e-4, optimizer_momentum=0.9)
moved = transform.optimize(save_transformed=True)

/home/rohitrango/code/CUDANTs/cudants/utils/imageutils.py:77: UserWarning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (Triggered internally at ../torch/csrc/jit/codegen/cuda/parser.cpp:3668.)
  image_smooth = separable_filtering(image, gaussians)
/home/rohitrango/code/CUDANTs/cudants/registration/affine.py:76: UserWarning: operator() profile_node %70 : int[] = prim::profile_ivalue[profile_failed="varying profile values"](%s.1)
 does not have profile information (Triggered internally at ../torch/csrc/jit/codegen/cuda/graph_fuser.cpp:105.)
  moving_image_blur = separable_filtering(moving_arrays, gaussians)
scale: 1, iter: 19/20, loss: -0.042985: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 24.65it/s]


In [7]:
print(transform.get_affine_matrix())

tensor([[[ 1.0263,  0.0220, -0.0119, -0.0215],
         [-0.0564,  0.9353,  0.0123, -0.0437],
         [ 0.0214, -0.0476,  0.9355,  0.0076],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]], device='cuda:0',
       grad_fn=<CatBackward0>)


In [8]:
# browse_image_slices(moved[-1][0, 0].data.cpu().numpy() - img1.array[0, 0].data.cpu().numpy())
browse_image_slices(moved[-1][0, 0].data.cpu().numpy())
browse_image_slices(img1.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

In [9]:
browse_image_slices(moved[-1][0, 0].data.cpu().numpy() - img2.array[0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

In [12]:
from cudants.registration.logdemons import LogDemonsRegistration
reg = LogDemonsRegistration([4, 2, 1], [50, 50, 50], fixed, moving, 
# reg = LogDemonsRegistration([1], [100], fixed, moving,
                eps_prime=0.5, init_affine=transform.get_affine_matrix(),  
                optical_flow_method='gauss-newton', update_sigma=0.25,
                                optical_flow_sigma=0.5, use_lie_bracket=False, symmetric=False)
saved_deformable = reg.optimize(save_transformed=True)

  0%|                                                                                                                                                                | 0/50 [00:00<?, ?it/s]

tensor(0., device='cuda:0') tensor(0., device='cuda:0')



scale: 4, iter: 0/50, loss: 965.251282:   2%|██▏                                                                                                             | 1/50 [00:02<01:40,  2.04s/it]

tensor(-0.0040, device='cuda:0') tensor(0.0041, device='cuda:0')



scale: 4, iter: 1/50, loss: 897.021118:   4%|████▍                                                                                                           | 2/50 [00:04<01:40,  2.09s/it]

...

tensor(-0.0118, device='cuda:0') tensor(0.0122, device='cuda:0')



scale: 4, iter: 3/50, loss: 790.421265:   8%|████████▉                                                                                                       | 4/50 [00:05<00:55,  1.20s/it]

tensor(-0.0155, device='cuda:0') tensor(0.0162, device='cuda:0')


scale: 4, iter: 3/50, loss: 790.421265:   8%|████████▉                                                                                                       | 4/50 [00:07<01:25,  1.86s/it]


KeyboardInterrupt: Interrupted by user

In [11]:
# browse_image_slices(saved_deformable[-1][0, 0].data.cpu().numpy()- moved[-1][0, 0].data.cpu().numpy())
browse_image_slices(saved_deformable[-1][0, 0].data.cpu().numpy())
browse_image_slices(img1.array[0, 0].data.cpu().numpy())
browse_image_slices(img2.array[0, 0].data.cpu().numpy())
# browse_image_slices(moved[-1][0, 0].data.cpu().numpy())

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

interactive(children=(IntSlider(value=77, description='x', layout=Layout(width='600px'), max=154), IntSlider(v…

## Check Optical Flow

In [ ]:
browse_image_slices(reg.velocity_fields[-1][0].data.cpu().numpy() - reg.velocity_fields[-1].min().item())